In [ ]:
import os
from openai import OpenAI
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from key import OPENAI_API_KEY

In [ ]:
#Configuration de l'API OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# Téléchargement des ressources NLTK
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [21]:
# Initialisation des analyseurs de sentiment
nltk_analyzer = SentimentIntensityAnalyzer()
transformer_analyzer = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [22]:
# Fonction de génération avec GPT-3
def generate_text_davinci(prompt, max_tokens=100):
    """Génère du texte avec GPT-3 (text-davinci-003)"""
    try:
        response = client.completions.create(
            model="text-davinci-003",
            prompt=prompt,
            max_tokens=max_tokens
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Erreur lors de la génération (davinci): {str(e)}"

In [23]:
# Fonction de génération avec GPT-4
def generate_text_gpt4(prompt, max_tokens=100):
    """Génère du texte avec GPT-4"""
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Erreur lors de la génération (GPT-4): {str(e)}"

In [24]:
#Fonction d'analyse de sentiment avec NLTK
def analyze_sentiment_nltk(text):
    """Analyse le sentiment avec NLTK VADER"""
    scores = nltk_analyzer.polarity_scores(text)
    return {
        'compound': scores['compound'],
        'positive': scores['pos'],
        'neutral': scores['neu'],
        'negative': scores['neg']
    }

In [25]:
#Fonction d'analyse de sentiment avec Transformers
def analyze_sentiment_transformers(text):
    """Analyse le sentiment avec le modèle Transformers"""
    result = transformer_analyzer(text)[0]
    return {
        'label': result['label'],
        'score': result['score']
    }


In [26]:


prompt = "Écrivez un avis sur un restaurant gastronomique."

print("=== Génération de texte avec GPT-3 ===")
davinci_text = generate_text_davinci(prompt)
print(davinci_text)
print("\n=== Analyse de sentiment NLTK ===")
print(analyze_sentiment_nltk(davinci_text))
print("\n=== Analyse de sentiment Transformers ===")
print(analyze_sentiment_transformers(davinci_text))


=== Génération de texte avec GPT-3 ===
Erreur lors de la génération (davinci): Error code: 404 - {'error': {'message': 'The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

=== Analyse de sentiment NLTK ===
{'compound': -0.4019, 'positive': 0.0, 'neutral': 0.906, 'negative': 0.094}

=== Analyse de sentiment Transformers ===
{'label': 'Very Negative', 'score': 0.6012861728668213}


In [27]:

print("=== Génération de texte avec GPT-4 ===")
gpt4_text = generate_text_gpt4(prompt)
print(gpt4_text)
print("\n=== Analyse de sentiment NLTK ===")
print(analyze_sentiment_nltk(gpt4_text))
print("\n=== Analyse de sentiment Transformers ===")
print(analyze_sentiment_transformers(gpt4_text))

=== Génération de texte avec GPT-4 ===
Restaurant: L'Envolée Gourmande

Dîner à L'Envolée Gourmande a été une expérience sensationnelle de gastronomie française. Edifié dans un cadre romantique et luxueux, ce restaurant conjugue tous les éléments pour passer un moment à la fois délicieux et inoubliable. 

Le service est impeccable. Le personnel est à la fois discret, attentif et toujours prêt à conseiller des accords mets

=== Analyse de sentiment NLTK ===
{'compound': 0.0, 'positive': 0.0, 'neutral': 1.0, 'negative': 0.0}

=== Analyse de sentiment Transformers ===
{'label': 'Very Positive', 'score': 0.6628829836845398}
